# Training a Neural Net in Julia

(from someone who has never trained a neural net before...

and never used Julia before the last few weeks...

and who is new to automatic differentiation...

and who knows shamefully little about machine learning)

In [1]:
using Pkg
Pkg.add("Flux")
using Flux

   Updating registry at `C:\Users\chris\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\chris\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\chris\.julia\environments\v1.5\Manifest.toml`


**Flux.jl**

- written entirely in Julia (convenient for people like me who get frustrated with external dependencies)

- easily readable mathematical syntax

- pretty well-documented, especially Julia (`https://fluxml.ai/`)

## Automatic Differentiation with `gradient`

In [3]:
# functions inline
f(x) = x^3 + x + 1

# f'(x) = 3x^2 + 1
df(x) = gradient(f,x)[1]
df(2)

13

In [4]:
# f''(x) = 6x
ddf(x) = gradient(df,x)[1]
ddf(2)

12

In [5]:
# continuous but not differentiable
g(x) = abs(x)
dg(x) = gradient(g,x)[1]

println(dg(-1), "  ", dg(0), "  ", dg(1))

-1  0  1


In [6]:
# multiple variables
f(x,y,z) = x^2 + cos(y) + z

# differentiating wrt 3 variable - > returns 3-tuple with derivative wrt each variable in order
df(x,y,z) = gradient(f,x,y,z) # df = (2x, -sin(y), 1)
df(1,0,0)


(2.0, -0.0, 1.0)

In [10]:
# mutiple variabels vectorized 
F(x) = x[1]^2 + cos(x[2]) + x[3]
x_vec = (1.,0.,0.)

# differentiating F with respect to vector x which has 3 components
# differentiating wrt 1 variabel which has 3 components
∇F(x) = gradient(F,x)[1]
∇F(x_vec)

(2.0, -0.0, 1.0)

## Example: Linear Regression

Idea: have a vector of observations $y$

Ansatz: $y = Wx + b$ where 

- features $x$ that you believe should be able to describe the observations

- weights $W$ which describe how the features contribute to the observations

- noise $b$

Goal: tune $W$ so that $x$ describes $y$ as well as possible (in view of some error function)

In [13]:
#L2 error
function loss(x,y,W,b)
   y1 = W*x+b
   return sum((y - y1).^2)
end

y = rand(5) # observations
x = rand(10) # parameter values

W = rand(5,10) #random initialization of weights
b = rand(5)    # random noise

println("Epoch ",0,"  Loss = ", loss(x,y,W,b)) # large loss because W is not tuned

alpha = 0.01 # learning rate

for i in 1:20

    #differentiating wrt 1 variable W which has 50 components
    ∂L_W = gradient(W -> loss(x,y,W,b),W)[1]
    W .-= alpha*∂L_W;
    println("Epoch ",i,"  Loss = ", loss(x,y,W,b)) # should decrease at each i
    
end

Epoch 0  Loss = 62.820077480585525
Epoch 1  Loss = 53.3614985659335
Epoch 2  Loss = 45.32706171975232
Epoch 3  Loss = 38.50233931507083
Epoch 4  Loss = 32.70518927298668
Epoch 5  Loss = 27.780893951116408
Epoch 6  Loss = 23.598030951028235
Epoch 7  Loss = 20.044965642414404
Epoch 8  Loss = 17.02687179449039
Epoch 9  Loss = 14.46320079953464
Epoch 10  Loss = 12.285531945765165
Epoch 11  Loss = 10.435746366411003
Epoch 12  Loss = 8.864475930291325
Epoch 13  Loss = 7.529785677010342
Epoch 14  Loss = 6.396054632848072
Epoch 15  Loss = 5.4330251379240035
Epoch 16  Loss = 4.614995312535394
Epoch 17  Loss = 3.9201331107519666
Epoch 18  Loss = 3.329893654338579
Epoch 19  Loss = 2.8285242964817003
Epoch 20  Loss = 2.4026442061785493


## Chaining Layers Together

At a formal level, a neural net is several of these linear regressions chained together, perhaps with some sort of activations functions in between (smoothing, thresholding,etc)

$$x \to y_1 \to y_2 \to \cdots \to y_n =: y$$

$$y_{k+1} = \sigma(W_k y_k + b_k)$$

## First idea: chain together manually

In [16]:
# interlayer activation function
function sig(x)
   return 1/(1 + exp(-x)) 
end

# L2 error
function loss2(x,y,W1,b1,W2,b2)
   y1 = sig.(W1*x+b1)
    y2 = W2*y1 + b2
    return sum((y-y2).^2)
end

y = rand(5) # observations
x = rand(10) # parameter values

W1 = rand(8,10) #random initialization of weights
b1 = rand(8)    # random noise

W2 = rand(5,8) #random initialization of weights
b2 = rand(5)    # random noise

println("Epoch ",0,"  Loss = ", loss2(x,y,W1,b1,W2,b2)) # large loss because W1,W2 are not tuned

alpha = 0.01 # learning rate

for i in 1:20
    ∂L_W1 = gradient((W1,W2) -> loss2(x,y,W1,b1,W2,b2),W1,W2)[1]
    ∂L_W2 = gradient((W1,W2) -> loss2(x,y,W1,b1,W2,b2),W1,W2)[2]
    W1 .-= alpha*∂L_W1
    W2 .-= alpha*∂L_W2

    println("Epoch ",i,"  Loss = ", loss2(x,y,W1,b1,W2,b2)) # should decrease for each i
    
end

Epoch 0  Loss = 82.52769127098489
Epoch 1  Loss = 59.34984778721228
Epoch 2  Loss = 42.71971783399389
Epoch 3  Loss = 30.7699156347399
Epoch 4  Loss = 22.17345785572703
Epoch 5  Loss = 15.984102847213755
Epoch 6  Loss = 11.525109614536937
Epoch 7  Loss = 8.31133092869889
Epoch 8  Loss = 5.994336386777744
Epoch 9  Loss = 4.323546024873815
Epoch 10  Loss = 3.1185741578551136
Epoch 11  Loss = 2.249475897157897
Epoch 12  Loss = 1.6225973539183354
Epoch 13  Loss = 1.1704180867184704
Epoch 14  Loss = 0.8442480618346297
Epoch 15  Loss = 0.6089711327160122
Epoch 16  Loss = 0.43925862439075625
Epoch 17  Loss = 0.31684042895709463
Epoch 18  Loss = 0.2285375563029926
Epoch 19  Loss = 0.16484333446209495
Epoch 20  Loss = 0.11890010685766617


 This will get murky after a few layers. All these things can be done using Flux commands
 
 The commands that will be really useful to us: `Dense`, `Chain` and `train!`

Flux.train! is called via:

Flux.train!(loss,params,data,optimizer,...)

- loss: the loss function you are minimizing
- params: the parameters you are minimizing wrt
- data: tuple (x,y) - the data the model is being fit to. Input x should result in output y
- optimizer: optimization routine (e.g. gradient descent)

In [17]:
using Base.Iterators: repeated
using Flux: throttle

In [20]:
# Chain command simply does functional composition (read left to right)
m = Chain(x-> x+3, x->x^2) # m(x) = (x+3)^2


25

In [31]:
# Dense creates layers exactly as above: y = σ(Wx+b)
# input dimensions of W and activation function σ
d1 = Dense(10,8,sig) # note order: Dense(10,8) creates a function that maps R^10 → R^8, W is a 8x10 matrix
d2 = Dense(8,6,sig)      # if you don't specify σ then σ = identity
d3 = Dense(6,5)

# compose the layers with chain
M = Chain(d1,d2,d3)

# loss function - M is left implicit
function L(x,y)
   return sum((M(x)-y).^2) 
end

# data
y = rand(5) # observations
x = rand(10) # parameter values
data = repeated((x,y),2000)

alpha = 0.01;
optimizer = Descent(alpha);

println(L(x,y))
# to get it to talk to  you, use callback "cb" in the fifth argument
Flux.train!(L,params(M),data,optimizer)#,cb = ()-> println(L(x,y)) )
println(L(x,y))
println(M(x))
println(convert(Array{Float32,1},y))

3.4704219493257993
1.7641846909872102e-13
Float32[0.17826554, 0.9829122, 0.4072043, 0.72196996, 0.66577137]
Float32[0.17826527, 0.9829123, 0.40720448, 0.7219702, 0.66577137]


# Example: MNIST

In [35]:
# import data and preprocessing
IMAGES = Flux.Data.MNIST.images()
LABELS = Flux.Data.MNIST.labels()

IMAGES[42]

In [34]:
LABELS[42]

8

In [38]:
using Statistics
using Flux: onehotbatch, onecold, crossentropy

#preprocessing
# make each image a long vector (28^2 length)
X = hcat(float.(reshape.(IMAGES[1:10000],:))...)
# labels are turned into binary vectors
Y = onehotbatch(LABELS[1:10000],0:9);
X[(1:28).+9*28,42]
Y[:,42]

10-element Flux.OneHotVector:
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0

In [41]:
# relu = threshold at 0, makes sure the yk vectors stay positive
# softmax = ensure that the output is a probability distribution
mnist_model = Chain(Dense(28^2,32,relu),Dense(32,10),softmax)

# L2 error is notoriously bad for comparing probability distributions; use crossentropy instead
loss_mnist(x,y) = crossentropy(mnist_model(x),y) 

opt_mnist = ADAM(); # stochastic gradient descent variant

data_mnist = repeated((X,Y),200);

In [44]:
# throttle(FUNC,n) - evaluate the function FUNC at most once every n second
Flux.train!(loss_mnist,params(mnist_model),data_mnist,opt_mnist,cb = throttle(() -> println(loss_mnist(X,Y)),10))

2.365936
1.8739222
1.4469699
1.1063274
0.865213
0.7045845
0.5976866
0.5243076
0.47211236
0.43343097
0.4034958
0.37949163
0.35967174
0.34290084
0.32842764
0.3172225
0.3057395
0.2953857
0.28594443
0.27728117
0.269267
0.26180407
0.25480962
0.24821705
0.2419625
0.23601013


There is way more functionality: activation functions, loss functions, accuracy functions, different types of layers (recurrent, convolutions, etc), etc

Documentation and basic examples at `fluxml.ai/Flux,jl/stable` 

Fully functional code at `github.com/FluxML/model-zoo`

## Test Accuracy

In [45]:
testIMAGES = Flux.Data.MNIST.images(:test)
testLABELS = Flux.Data.MNIST.labels(:test)
testX = hcat(float.(reshape.(testIMAGES,:))...)
testY = onehotbatch(testLABELS,0:9);

In [46]:
testIMAGES[8128]

In [47]:
mnist_model(testX[:,8128])

10-element Array{Float32,1}:
 0.9991473
 6.794925f-10
 9.983294f-6
 5.9521767f-7
 8.859609f-8
 0.0006252304
 1.9617755f-5
 3.343224f-7
 0.00019033856
 6.4854644f-6

In [48]:
onecold(mnist_model(testX[:,8128]))-1 

0

In [49]:
accuracy(x,y) = mean(onecold(mnist_model(x)).== onecold(y));
accuracy(X,Y)

0.9389

In [50]:
accuracy(testX,testY)

0.9182